In [30]:
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt
import os

In [5]:
adata_sp = sc.read_h5ad('adata.h5ad')
print(adata_sp)

C:\Users\Roulis_Lab\AppData\Roaming\Python\Python312\site-packages\anndata\_core\aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 1348582 × 990
    obs: 'x', 'y', 'Mouse_ID', 'Technical_repeat_number', 'Sample_type', 'Slice_ID', 'FOV', 'Cell_ID', 'sample', 'N_genes', 'Tier1', 'Tier2', 'Tier3', 'Leiden_neigh', 'Neigh_umap_x', 'Neigh_umap_y', 'Tier2_umap_x', 'Tier2_umap_y', 'Tier3_umap_x', 'Tier3_umap_y', 'Tier1_umap_x', 'Tier1_umap_y'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Neigh_anndata', 'Sample_type_colors', 'Tier1_colors', 'Tier2_colors', 'Tier3_colors', 'hvg', 'leiden', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'


In [6]:
adata = sc.read_h5ad('combined_mouse_colon.h5ad')
print(adata)

AnnData object with n_obs × n_vars = 7181 × 16693
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'RNA_snn_res.0.5', 'seurat_clusters', 'doublet_class', 'Epithelial_sum_expr', 'PanImmune_sum_expr', 'PlasmaCell_sum_expr', 'NKCell_sum_expr', 'Myeloid_sum_expr', 'Monocyte_Mac_sum_expr', 'Dendritic_sum_expr', 'MastCell_sum_expr', 'Endothelial_sum_expr', 'Lymphatic_Endo_sum_expr', 'Glial_sum_expr', 'Pericyte_sum_expr', 'SMC_sum_expr', 'Fibroblast_sum_expr', 'cell_type', 'condition', 'mouse', 'Epithelial_marker_sum', 'PanImmune_marker_sum', 'PlasmaCell_marker_sum', 'NKCell_marker_sum', 'Myeloid_marker_sum', 'Monocyte_Mac_marker_sum', 'Dendritic_marker_sum', 'MastCell_marker_sum', 'Endothelial_marker_sum', 'Lymphatic_Endo_marker_sum', 'Glial_marker_sum', 'Pericyte_marker_sum', 'SMC_marker_sum', 'Fibroblast_marker_sum', 'predicted.id', 'prediction.score.SMC', 'prediction.score.Lymphatic.endothelial', 'prediction.score.Fibroblasts', 'prediction.score.Vascular.endothelial', 'p

In [13]:
common_genes = sorted(adata_sp.var_names.intersection(adata.var_names))
print(f"{len(common_genes)} genes in common")

760 genes in common


In [16]:
adata_sp = adata_sp[:, common_genes].copy()
adata    = adata[:, common_genes].copy()
print(adata_sp.n_vars == adata.n_vars)

True


In [18]:
sc.pp.pca(adata_sp, n_comps= 50, svd_solver='arpack')
sc.pp.neighbors(adata_sp, n_neighbors=15, n_pcs=50)
sc.pp.pca(adata, n_comps=50, svd_solver="arpack")

C:\Users\Roulis_Lab\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# Create a spatial embedding in the reference AnnData
coords_ref = adata_sp.obs[['x','y']].to_numpy()
adata_sp.obsm['spatial'] = coords_ref      

print(adata_sp.obsm['spatial'].shape)

(1348582, 2)


In [29]:
sc.tl.ingest(adata, adata_sp, obs='Tier1', embedding_method=['pca', 'umap'])

C:\Users\Roulis_Lab\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [36]:
idx = adata_sp.obs_names.get_indexer(adata.obs_names)
coords = adata_sp.obsm['spatial'][idx, :]
adata.obsm['spatial'] = coords

In [32]:
# Define your cell-type markers
celltype_markers = {
  "Epithelial":       ["Cdh1","Epcam","Alpi","Muc2","Vil1","Lyz1","Lgr5","Olfm4","Ccnd1"],
  "PanImmune":        ["Ptprc"],
  "BCell":            ["Ms4a1","Cd19"],
  "PlasmaCell":       ["Mzb1","Ccr10","Sdc1"],
  "TCell":            ["Cd3e"],
  "CD4_TCell":        ["Cd4"],
  "CD8_TCell":        ["Cd8a","Cd8b"],
  "NKCell":           ["Nkg7","Ncam1"],
  "Myeloid":          ["Cd68","Cd74"],
  "Monocyte_Mac":     ["Cd14"],
  "Dendritic":        ["Itgax"],
  "MastCell":         ["Kit","Il1rl1"],
  "Treg":             ["Foxp3"],
  "Endothelial":      ["Pecam1"],
  "Lymphatic_Endo":   ["Lyve1"],
  "Glial":            ["Plp1","S100b"],
  "Pericyte":         ["Rgs5","Pdgfrb","Adipoq"],
  "SMC":              ["Myh11","Actg2","Myocd","Des","Acta2"],
  "Fibroblast":       ["Pdgfra","Sparc","Dcn","Lum","Col1a1","Col14a1","Fgfr2",
                       "Col3a1","Col4a5","Col4a6","Bmp5","Bmp4","Grem1","Vcam1",
                       "Ogn","Mgp","Sfrp2","C3","Dpt"]
}


In [37]:
out_dir = "SpatialPlots"
os.makedirs(out_dir, exist_ok=True)

# loop markers, plot HC vs DSS side by side
for celltype, markers in celltype_markers.items():
    for gene in markers:
        if gene not in adata.var_names:
            continue
        fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
        # HC
        sc.pl.spatial(
            adata[adata.obs["condition"] == "HC"],
            color       = gene,
            ax          = axes[0],
            title       = f"HC — {gene}",
            spot_size   = 30,
            show        = False
        )
        # DSS
        sc.pl.spatial(
            adata[adata.obs["condition"] == "DSS"],
            color       = gene,
            ax          = axes[1],
            title       = f"DSS — {gene}",
            spot_size   = 30,
            show        = False
        )
        fig.tight_layout()
        # save into SpatialPlots/
        save_path = os.path.join(out_dir, f"Spatial_{celltype}_{gene}.png")
        fig.savefig(save_path, dpi=300)
        plt.close(fig)


C:\Users\Roulis_Lab\AppData\Local\Temp\ipykernel_22164\1493527844.py:11: FutureWarning: Use `squidpy.pl.spatial_scatter` instead.
  sc.pl.spatial(
C:\Users\Roulis_Lab\AppData\Local\Temp\ipykernel_22164\1493527844.py:20: FutureWarning: Use `squidpy.pl.spatial_scatter` instead.
  sc.pl.spatial(


In [38]:
adata_sp.write('adata_copy.h5ad')
adata.write('combined_copy.h5ad')